In [2]:
import cv2
import mediapipe as mp
import joblib
import numpy as np
from collections import Counter
from pathlib import Path


In [3]:
ngt_static_classifier = Path('./data/models/ngt_static_classifier_normalized.pkl')

# Load the NORMALIZED model
clf = joblib.load(ngt_static_classifier)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def normalize_landmarks(landmarks):
    """Normalize landmarks - must match training normalization"""
    coords = np.array([[lm.x, lm.y, lm.z] for lm in landmarks])
    
    # Center on wrist (landmark 0)
    wrist = coords[0]
    coords = coords - wrist
    
    # Scale by hand size (wrist to middle finger tip)
    hand_size = np.linalg.norm(coords[12] - coords[0])
    if hand_size > 0:
        coords = coords / hand_size
    
    return coords.flatten()

print("Starting real-time NGT recognition with normalized model...")
print("Press 'q' to quit\n")

cap = cv2.VideoCapture(0)
prediction_buffer = []
buffer_size = 5

try:
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7
    ) as hands:
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb_frame)
            
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS
                    )
                    
                    # Normalize landmarks
                    normalized_coords = normalize_landmarks(hand_landmarks.landmark)
                    
                    # Predict
                    prediction = clf.predict([normalized_coords])[0]
                    probabilities = clf.predict_proba([normalized_coords])[0]
                    confidence = probabilities.max()
                    
                    # Smooth predictions
                    prediction_buffer.append(prediction)
                    if len(prediction_buffer) > buffer_size:
                        prediction_buffer.pop(0)
                    
                    smoothed = Counter(prediction_buffer).most_common(1)[0][0]
                    
                    # Display
                    cv2.putText(frame, f"Letter: {smoothed}", 
                               (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
                    cv2.putText(frame, f"Confidence: {confidence:.1%}", 
                               (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "No hand detected", 
                           (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                prediction_buffer = []
            
            cv2.imshow('NGT Recognition', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

finally:
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.waitKey(1)

print("\nWindow closed")

Starting real-time NGT recognition with normalized model...
Press 'q' to quit



I0000 00:00:1768852120.453846  685243 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1768852120.472309  687129 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768852120.477785  687129 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768852121.502569  687132 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.



Window closed
